In [13]:
import random
import pickle as cPickle
import numpy as np
import math
import pandas
import tensorflow as tf
from sklearn.cluster import *
from sklearn import preprocessing

tf.reset_default_graph()
random.seed(2016)
sampleNum = 10

def completeTrajectories():
    path = './X.npy'
    x = np.load(path)
    print("===========")
    print(x.shape[0])
    print("===========")
    #top5 = x[:2]
    #print(top5)
    #seq = x[1]
    #print(seq)
    #print(seq.shape[0])
    #simTrjss = cPickle.load(open('./simulated_data/sim_trajectories','rb'))
    simTrjComps = []
    simTrjss = []
    simSeq = []
    numJoints=20
    #dim=3
    allX = []
    allY = []
    allZ = []
    joints = []
    N = 3
    
    for seq in x[:40]:
        print(seq)
        print("===========")
        for i in range(0, seq.shape[0]):
            #print(i)
            #print(seq[i])
            joints = []
            joints = [seq[i][n:n+N] for n in range(0, len(seq[i]), N)]
            print(joints)
            #joints.append(subList)
            simTrjss.append(joints)
        simSeq.append(simTrjss)
     
    
    print("BATATA===============")    
    simSeq_np = np.array(simSeq)
    print(simSeq_np)
    print("BATATA===============") 
    for sseq in simSeq_np:
        for simTrjs in sseq:
            trjsCom = []
            time = 0
            for i in range(0,len(simTrjs)):
                rec = []

                if i==0:
                    # time, locationC, speedC, rotC
                    rec = [time,0,0]#,0]
                else:
                    time = time + 5
                    locC = math.sqrt((simTrjs[i][0]-simTrjs[i-1][0])**2+(simTrjs[i][1]-simTrjs[i-1][1])**2+(simTrjs[i][2]-simTrjs[i-1][2])**2)
                    rec.append(time)
                    rec.append(locC)
                    #rec.append(locC/(simTrjs[i][0]-simTrjs[i-1][0]))
                    rec.append(locC/5)
                    #rec.append(math.atan((simTrjs[i][2]-simTrjs[i-1][2])/ (simTrjs[i][1]-simTrjs[i-1][1])))
                trjsCom.append(rec)
            simTrjComps.append(trjsCom)
    print("DONE=========")
    print(len(simTrjComps))
    cPickle.dump(simTrjComps,open('./simulated_data/sim_traj_humanMvt_complete','wb'))
    return simTrjComps

def computeFeas():
    simTrjCompss = cPickle.load(open('./simulated_data/sim_traj_humanMvt_complete','rb'))
    simTrjFeas = []
    for simTrjComps in simTrjCompss:
        trjsComfea = []
        for i in range(0,len(simTrjComps)):
            rec = []
            if i==0:
                # time, locationC, speedC, rotC
                rec = [0,0,0]#,0]
            else:
                locC = simTrjComps[i][1]
                locCrate = locC/(simTrjComps[i][0]-simTrjComps[i-1][0])
                rec.append(simTrjComps[i][0])
                rec.append(locCrate)
                if locCrate<3:
                    rec.append(0)
                    #rec.append(0)
                else:
                    rec.append(simTrjComps[i][2]-simTrjComps[i-1][2])
                    #rec.append(simTrjComps[i][3]-simTrjComps[i-1][3])
            trjsComfea.append(rec)
        simTrjFeas.append(trjsComfea)
    cPickle.dump(simTrjFeas, open('./simulated_data/sim_traj_humanMvt_feas', 'wb'))
    #print(simTrjFeas)
    return simTrjFeas

def rolling_window(sample, windowsize = 600, offset = 300):
    #print("ROLLING WINDOW")
    timeLength = sample[len(sample)-1][0]
    windowLength = int (timeLength/offset)+1
    windows = []
    for i in range(0,windowLength):
        windows.append([])

    for record in sample:
        time = record[0]
        for i in range(0,windowLength):
            if (time>(i*offset)) & (time<(i*offset+windowsize)):
                windows[i].append(record)
                
   # print(windows)
    return windows
    # pass

def behavior_ext(windows):
    #print("BEHAVIOR EXTTTTTTTTTTT")
    
    behavior_sequence = []
    for window in windows:
        behaviorFeature = []
        records = np.array(window)
        if len(records) != 0:
            # print np.shape(records)
            pd = pandas.DataFrame(records)
            pdd =  pd.describe()
            #print(pdd)
            # print pdd[1][0]
            # for ii in range(1,4):
            #     for jj in range(1,8):
            #         behaviorFeature.append(pdd[ii][jj])
            # behaviorFeature.append(pdd[0][1])
            behaviorFeature.append(pdd[1][1])
            behaviorFeature.append(pdd[2][1])
            #behaviorFeature.append(pdd[3][1])
            # behaviorFeature.append(pdd[0][2])
            # behaviorFeature.append(pdd[1][2])
            # behaviorFeature.append(pdd[2][2])
            # behaviorFeature.append(pdd[3][2])
            # behaviorFeature.append(pdd[0][3])
            behaviorFeature.append(pdd[1][3])
            behaviorFeature.append(pdd[2][3])
            #behaviorFeature.append(pdd[3][3])
            # behaviorFeature.append(pdd[0][4])
            behaviorFeature.append(pdd[1][4])
            behaviorFeature.append(pdd[2][4])
            #behaviorFeature.append(pdd[3][4])
            # behaviorFeature.append(pdd[0][5])
            behaviorFeature.append(pdd[1][5])
            behaviorFeature.append(pdd[2][5])
            #behaviorFeature.append(pdd[3][5])
            # behaviorFeature.append(pdd[0][6])
            behaviorFeature.append(pdd[1][6])
            behaviorFeature.append(pdd[2][6])
            #behaviorFeature.append(pdd[3][6])
            # behaviorFeature.append(pdd[0][7])
            behaviorFeature.append(pdd[1][7])
            behaviorFeature.append(pdd[2][7])
            #behaviorFeature.append(pdd[3][7])

            behavior_sequence.append(behaviorFeature)
    #print(behavior_sequence)
    return behavior_sequence

def generate_behavior_sequences():
    f = open('./simulated_data/sim_traj_humanMvt_feas','rb')
    sim_data = cPickle.load(f)
    behavior_sequences = []

    for sample in sim_data:
        #print(sample)
        windows = rolling_window(sample)
        behavior_sequence = behavior_ext(windows)
        print(len(behavior_sequence))
        behavior_sequences.append(behavior_sequence)
    fout = open('./simulated_data/sim_behavior_humanMvt_sequences','wb')
    cPickle.dump(behavior_sequences,fout)

def generate_normal_behavior_sequence():
    print("generate_normal_behavior_sequence")
    f = open('./simulated_data/sim_behavior_humanMvt_sequences','rb')
    behavior_sequences = cPickle.load(f)
    print(behavior_sequences)
    print(np.shape(behavior_sequences))
    behavior_sequences_normal = []
    templist = []
    for item in behavior_sequences:
        for ii in item:
            templist.append(ii)
        #print (len(item))
    #print(len(templist))
    min_max_scaler = preprocessing.MinMaxScaler()
    # print np.shape(behavior_sequence)
    templist_normal = min_max_scaler.fit_transform(templist).tolist()
    index = 0
    for item in behavior_sequences:
        behavior_sequence_normal = []
        for ii in item:
            behavior_sequence_normal.append(templist_normal[index])
            index = index + 1
        #print(len(behavior_sequence_normal))
        behavior_sequences_normal.append(behavior_sequence_normal)
    #print(index)
    print("generate_normal_behavior_sequence222222222222222")
    print(behavior_sequences_normal)
    fout = open('./simulated_data/sim_normal_behavior_humanMvt_sequences', 'wb')
    cPickle.dump(behavior_sequences_normal, fout)


    
def trajectory2Vec():
    def loopf(prev, i):
        return prev

    # Parameters
    learning_rate = 0.0001
    training_epochs = 300
    display_step = 100

    # Network Parameters
    # the size of the hidden state for the lstm (notice the lstm uses 2x of this amount so actually lstm will have state of size 2)
    size = 100
    # 2 different sequences total
    batch_size = 1
    # the maximum steps for both sequences is 5
    max_n_steps = 17
    # each element/frame of the sequence has dimension of 3
    frame_dim = 12 #18

    input_length = tf.placeholder(tf.int32)

    initializer = tf.random_uniform_initializer(-1, 1)

    # the sequences, has n steps of maximum size
    # seq_input = tf.placeholder(tf.float32, [batch_size, max_n_steps, frame_dim])
    seq_input = tf.placeholder(tf.float32, [max_n_steps, batch_size, frame_dim])
    # what timesteps we want to stop at, notice it's different for each batch hence dimension of [batch]

    # inputs for rnn needs to be a list, each item/frame being a timestep.
    # we need to split our input into each timestep, and reshape it because split keeps dims by default

    useful_input = seq_input[0:input_length[0]]
    loss_inputs = [tf.reshape(useful_input, [-1])]
    encoder_inputs = [item for item in tf.unstack(seq_input)]
    # if encoder input is "X, Y, Z", then decoder input is "0, X, Y, Z". Therefore, the decoder size
    # and target size equal encoder size plus 1. For simplicity, here I droped the last one.
    decoder_inputs = ([tf.zeros_like(encoder_inputs[0], name="GO")] + encoder_inputs[:-1])
    targets = encoder_inputs

    # basic LSTM seq2seq model
    cell = tf.nn.rnn_cell.LSTMCell(size, state_is_tuple=True, use_peepholes=True)
    _, enc_state = tf.contrib.rnn.static_rnn(cell, encoder_inputs, sequence_length=input_length[0], dtype=tf.float32)
    cell = tf.contrib.rnn.OutputProjectionWrapper(cell, frame_dim)
    dec_outputs, dec_state = tf.contrib.legacy_seq2seq.rnn_decoder(decoder_inputs, enc_state, cell, loop_function=loopf)


    # flatten the prediction and target to compute squared error loss
    y_true = [tf.reshape(encoder_input, [-1]) for encoder_input in encoder_inputs]
    y_pred = [tf.reshape(dec_output, [-1]) for dec_output in dec_outputs]

    # Define loss and optimizer, minimize the squared error
    loss = 0
    for i in range(len(loss_inputs)):
        loss += tf.reduce_sum(tf.square(tf.subtract(y_pred[i], y_true[len(loss_inputs) - i - 1])))
    optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

    # Initializing the variables
    init = tf.initialize_all_variables()

    # Launch the graph
    with tf.Session() as sess:
        sess.run(init)
        # Training cycle
        input_datas = cPickle.load(open('./simulated_data/sim_normal_behavior_humanMvt_sequences','rb'))
        print("INPUT DATAS")
        print(input_datas)
        trajectoryVecs = []
        j = 0
        for input_data in input_datas:
            print ('Sample:')
            print (j)
            input_len = len(input_data)
            print (input_len)
            defalt = []
            for i in range(0, frame_dim):
                defalt.append(0)
            while len(input_data) < max_n_steps:
                input_data.append(defalt)
            x = np.array(input_data)
            print("shape======")
            print (np.shape(x[0]))
            print(x)
            x = x.reshape((max_n_steps, batch_size, frame_dim))
            embedding = None
            for epoch in range(training_epochs):
                feed = {seq_input: x, input_length: np.array([input_len])}
                # Fit training using batch data
                _, cost_value, embedding, en_int, de_outs, loss_in = sess.run(
                    [optimizer, loss, enc_state, encoder_inputs, dec_outputs, loss_inputs], feed_dict=feed)
                # Display logs per epoch step
                if epoch % display_step == 0:
                    print ("logits")
                    a = sess.run(y_pred, feed_dict=feed)
                    print ("labels")
                    b = sess.run(y_true, feed_dict=feed)

                    print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(cost_value))
            trajectoryVecs.append(embedding)
            print("Optimization Finished!")
            j = j + 1
        fout = open('./simulated_data/sim_traj_vec_normal_humanMvt_reverse', 'wb')
        cPickle.dump(trajectoryVecs, fout)

def vecClusterAnalysis():
    print ('---------------------------------')
    print ('Our Method')
    trVecs = []
    trs = cPickle.load(open('./simulated_data/sim_traj_vec_normal_humanMvt_reverse','rb'))
    inte = []
    for tr in trs:
        print("-----------")
        print(tr)
        trVecs.append(tr[0][0])
        
    print("TRRR")
    print(tr)
    km = KMeans(n_clusters=3, random_state=2016)
    clusters = km.fit(trVecs).labels_.tolist()
    print("clusters")
    print(clusters)
    all = 0.
    print("###")
    print(clusters[:sampleNum])
    item = set(clusters[:sampleNum])
    print("itemmmm")
    print(item)
    l = []
    for i in item:
        l.append([i,clusters[:sampleNum].count(i)])
        print('Straight dans for:  '+ str(l))
        
    print ('Straight:  '+ str(l))
    m = max([te[1] for te in l])
    print("MAXXX",m)
    all = all + m
    print (float(m)/sampleNum)


    m = 0.
    item = set(clusters[sampleNum:sampleNum*2])
    l = []
    for i in item:
        l.append([i,clusters[sampleNum:sampleNum*2].count(i)])
    print ('Circling:  '+ str(l))
    m = max([te[1] for te in l])
    all = all + m
    print (float(m)/sampleNum)

    m = 0.
    item = set(clusters[sampleNum*2:sampleNum*3])
    l = []
    for i in item:
        l.append([i,clusters[sampleNum*2:sampleNum*3].count(i)])
    m = max([te[1] for te in l])
    print ('bending:   '+ str(l))
    all = all + m
    print (float(m)/sampleNum)
    print ('overall')
    print (all/(sampleNum*3))
    print ('---------------------------------')


if __name__ == '__main__':
    completeTrajectories()
    computeFeas()
    generate_behavior_sequences()
    generate_normal_behavior_sequence()
    trajectory2Vec()
    print("finishhhhhhhhhh")
    #vecClusterAnalysis()

567
[[-0.35713251  0.36986165  2.68409998 ... -0.17262143  0.65932591
   2.54934823]
 [-0.37448502  0.36632716  2.67215094 ... -0.19072368  0.6569371
   2.54794216]
 [-0.37379072  0.36653253  2.67709913 ... -0.1846042   0.65669163
   2.54576767]
 ...
 [-0.28949237  0.36285201  2.76733008 ... -0.15130397  0.67986271
   2.59508208]
 [-0.29884544  0.36511623  2.75388486 ... -0.14562677  0.66912062
   2.58056398]
 [-0.27681412  0.38070862  2.75843502 ... -0.12432163  0.66845585
   2.57404473]]
[array([-0.35713251,  0.36986165,  2.68409998]), array([-1.66066020e-03,  3.54087896e-01,  2.67285988e+00]), array([-0.18035343,  0.40757698,  2.5846011 ]), array([-0.18325419,  0.20669618,  2.77500737]), array([-0.3237759 , -0.01468199,  2.65024112]), array([-0.04489828, -0.02417835,  2.62955171]), array([-0.17490352, -0.00324405,  2.76304837]), array([-0.41915032,  0.11045902,  2.67649601]), array([0.04940912, 0.0926925 , 2.67124733]), array([-0.40116335, -0.17275062,  2.62993544]), array([ 0.03736

[array([-0.3807274 ,  0.35929805,  2.81278179]), array([-0.09000138,  0.41140399,  2.61050881]), array([-0.29538208,  0.42451433,  2.6345648 ]), array([-0.15572836,  0.23797068,  2.78715285]), array([-0.32637603,  0.01737321,  2.73566265]), array([-0.09269509, -0.00286006,  2.58007386]), array([-1.40553173e-01,  1.80798946e-03,  2.76221111e+00]), array([-0.44773791,  0.11144637,  2.79029925]), array([-0.10142565,  0.43985728,  2.35685703]), array([-0.46148851, -0.16600642,  2.76663995]), array([-0.11596216,  0.59616944,  2.13017764]), array([-0.4557379 , -0.23714629,  2.77643063]), array([-0.1177038,  0.6460993,  2.0839765]), array([-0.34897012, -0.55796836,  2.7306958 ]), array([-0.10827569, -0.56614996,  2.6903245 ]), array([-0.35898399, -0.98188526,  2.78099104]), array([-0.08078695, -0.988999  ,  2.74296348]), array([-0.45800047, -0.99560423,  2.64526827]), array([-0.17514955, -1.01211543,  2.60517575]), array([-0.19272021,  0.68316095,  2.60179165])]
[array([-0.38496315,  0.361608

[array([-0.25991471,  0.07203549,  2.9668566 ]), array([-0.02346489,  0.24889342,  2.69720679]), array([-0.24868482,  0.22555542,  2.78366491]), array([0.00365975, 0.02393957, 2.86147031]), array([-0.24527695, -0.20319904,  2.87185341]), array([-0.07457001, -0.17206494,  2.64490385]), array([ 4.80863505e-04, -2.35825649e-01,  2.87897383e+00]), array([-0.23709084,  0.11729174,  2.70910834]), array([-0.03472883,  0.40558481,  2.47675942]), array([-0.20162228,  0.39399933,  2.68112831]), array([-0.07945296,  0.59948212,  2.24645641]), array([-0.19518159,  0.46340687,  2.6538471 ]), array([-0.1036623 ,  0.6597392 ,  2.19791771]), array([-0.0475872 , -0.68569687,  2.73013312]), array([-0.3212555 , -0.64258322,  2.78539491]), array([-0.0803475 , -1.07922877,  2.86633638]), array([-0.26216556, -1.05168225,  2.83371291]), array([-0.09491768, -1.13690471,  2.69303702]), array([-0.43830239, -1.0694631 ,  2.88422629]), array([-0.45051826,  0.15866611,  2.67461619])]
[array([-0.25526748,  0.058959

[array([-0.370262,  0.336608,  2.774969]), array([0.034175, 0.357359, 2.649367]), array([-0.204442,  0.425092,  2.607869]), array([-0.139208,  0.219765,  2.784001]), array([-0.32351 ,  0.062772,  2.69331 ]), array([0.003052, 0.065179, 2.582061]), array([-0.118407,  0.039532,  2.75992 ]), array([-0.417578,  0.05146 ,  2.722689]), array([0.191857, 0.216207, 2.445665]), array([-0.434933, -0.259142,  2.64721 ]), array([0.410952, 0.183373, 2.214601]), array([-0.388908, -0.324535,  2.667282]), array([0.45707 , 0.20545 , 2.149924]), array([-0.336429, -0.515443,  2.625867]), array([-0.054435, -0.505405,  2.721807]), array([ 0.085587, -0.691418,  2.652318]), array([-0.400496, -0.804167,  2.749125]), array([ 2.145000e-03, -8.927790e-01,  2.643976e+00]), array([-0.257761, -0.969026,  2.754313]), array([-0.201028,  0.666485,  2.621388])]
[array([-0.364042,  0.337258,  2.773316]), array([0.034934, 0.364383, 2.632396]), array([-0.207293,  0.42097 ,  2.595355]), array([-0.128617,  0.229403,  2.781229

[array([-0.434711,  0.229303,  2.505758]), array([-0.076113,  0.236848,  2.474925]), array([-0.264316,  0.279807,  2.39822 ]), array([-0.245031,  0.10214 ,  2.579041]), array([-0.447812, -0.119783,  2.471285]), array([-0.021914, -0.11196 ,  2.467158]), array([-0.233881, -0.105853,  2.589597]), array([-0.494695, -0.036811,  2.509576]), array([ 0.012465, -0.020931,  2.486483]), array([-0.464943, -0.310423,  2.421855]), array([ 0.027892, -0.30481 ,  2.435314]), array([-0.485198, -0.375137,  2.410794]), array([ 0.041317, -0.369892,  2.452763]), array([-0.352335, -0.617684,  2.53015 ]), array([-0.114394, -0.6169  ,  2.50647 ]), array([-0.355215, -1.002746,  2.622536]), array([-0.136032, -0.988306,  2.642137]), array([-0.359043, -1.041729,  2.459938]), array([-0.127918, -1.04604 ,  2.485378]), array([-0.261212,  0.524992,  2.377543])]
[array([-0.433939,  0.227745,  2.51    ]), array([-0.075623,  0.238928,  2.47707 ]), array([-0.264683,  0.280903,  2.401945]), array([-0.242646,  0.101621,  2.

...

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


KeyboardInterrupt: 